In [1]:
import graphviz
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import cython
import tweepy
import json
import requests
import inspect
import re
from tqdm import tqdm
from time import time, sleep
import networkx as nx

In [2]:
from ast import literal_eval

def load_data(extras = True):

    data = pd.read_csv("politicians_connections_two.csv")
    data["Following"] = data["Following"].apply(literal_eval)

    print(data.head())

    usernames = data["Username"].tolist()
    ids = data["ids"].tolist()
    
    return data, usernames, ids

data, usernames, ids = load_data()

        Party             Name Branch        Username                 ids  \
0  Republican    Pete Sessions  House    PeteSessions            24735461   
1  Democratic   Debbie Dingell  House   RepDebDingell          2970279814   
2  Democratic  Pramila Jayapal  House      RepJayapal  815733290955112448   
3  Republican      Rob Wittman  House      RobWittman            15356407   
4  Democratic  Chellie Pingree  House  chelliepingree            14984637   

                                           Following  
0  {'ids': [1396151678, 815966620300480514, 12912...  
1  {'ids': [353272992, 300463737, 393541673, 9873...  
2  {'ids': [35281549, 25535595, 19847765, 1002630...  
3  {'ids': [834069080, 2964877294, 3686482216, 29...  
4  {'ids': [38026753, 873135988440223745, 1462548...  


In [3]:
def get_client():

    with open("dev_keys_json.json", "r") as f:
        keys = json.load(f)
    client = tweepy.Client(bearer_token=keys["Bearer_token"],
                           access_token = keys["API_key"],
                           access_token_secret = keys["API_secret_key"],
                           wait_on_rate_limit=True)
    return client

    

def decompose_tweet(self,tweet):
    try:
        retweeted_status = tweet.retweeted_status
        is_retweet = True
        #print(self.count,is_retweet)
        #print(retweeted_status.user.id)
        #print(tweet.user.id)
    except:
        is_retweet = False
        #print(dir(tweet))
        #print("\n")
        self.decompose_tweet(tweet)
        pass
    if is_retweet:
        retweet = tweet
        tweet = tweet.retweeted_status
        #self.decompose_tweet(original)


    user  = tweet.user
    username = user.screen_name
    id = user.id
    text = tweet.text
    print(username, text)       
    


class Listener(tweepy.Stream):
    def __init__(self, keys, ids):
        
        os.chdir("./stream_out")
        
        self.count = 0
        
        super().__init__(keys["API_key"], keys["API_secret_key"],
                                keys["Access_token"], keys["Access_token_secret"])
        


    def on_status(self, tweet):

        #self.decompose_tweet(tweet)
        
        tweet_id = tweet.id
        
        print(tweet._json)
        
        with open(f"{tweet_id}.json", "w") as f:
  
            json.dump(tweet._json, f, indent = 6)
        

        self.count += 1
        #print(self.count,tweet.retweet, tweet.author)
        
        if self.count >= 3:
            self.disconnect()

def get_stream(ids):
    
    with open("dev_keys_api_one.json", "r") as f:
        keys = json.load(f)
    
    client = Listener(keys, ids)

    return client

stream = get_stream(ids)

#stream.filter(follow=ids)
stream.filter(languages=["en"], follow=ids, filter_level = "low")



{'created_at': 'Mon Mar 07 13:25:21 +0000 2022', 'id': 1500824930023264261, 'id_str': '1500824930023264261', 'text': 'RT @RepRalphNorman: Joe Biden’s gas crisis needs to end.', 'source': '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 218104966, 'id_str': '218104966', 'name': 'Karen Nelson', 'screen_name': 'dixonres', 'location': None, 'url': None, 'description': None, 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 234, 'friends_count': 90, 'listed_count': 1, 'favourites_count': 22239, 'statuses_count': 41434, 'created_at': 'Sun Nov 21 12:42:12 +0000 2010', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'C0DEED'

Stream connection closed by Twitter


{'created_at': 'Mon Mar 07 13:25:23 +0000 2022', 'id': 1500824934720888834, 'id_str': '1500824934720888834', 'text': '@RepMGriffith As the Lord said: “He that loveth not knoweth not God; for God is love”. Help us with at least anti-r… https://t.co/DgI46SrG5U', 'display_text_range': [14, 140], 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'truncated': True, 'in_reply_to_status_id': 1499874194900856837, 'in_reply_to_status_id_str': '1499874194900856837', 'in_reply_to_user_id': 234057152, 'in_reply_to_user_id_str': '234057152', 'in_reply_to_screen_name': 'RepMGriffith', 'user': {'id': 1500765716840071169, 'id_str': '1500765716840071169', 'name': 'Severyn Skaskiv', 'screen_name': 'esevska', 'location': None, 'url': None, 'description': None, 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 0, 'friends_count': 12, 'listed_count': 0, 'favourites_count': 0, 'statuses_count': 35, 'created_at': 'Mon Mar 07 09:30:20 +0000

In [4]:
print(stream.running)

False


In [5]:
stream.filter(follow=ids)

Stream connection closed by Twitter


{'created_at': 'Mon Mar 07 13:25:23 +0000 2022', 'id': 1500824937593978887, 'id_str': '1500824937593978887', 'text': 'RT @MarshaBlackburn: Joe Biden is reportedly set to give Iran $7 billion in frozen funds.', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1363567086484811776, 'id_str': '1363567086484811776', 'name': 'Rod Dixon', 'screen_name': 'RodDixo39614362', 'location': None, 'url': None, 'description': None, 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 103, 'friends_count': 712, 'listed_count': 0, 'favourites_count': 33432, 'statuses_count': 19769, 'created_at': 'Sun Feb 21 19:12:23 +0000 2021', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang': None, 'contributors_enabled': False